In [2]:
import pandas as pd
import numpy as np
import pandas.api.types as ptypes
import sklearn.decomposition


/Users/russell_a/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [4]:
df = pd.read_csv("./Egypt_Sample_Data.csv")

,Date,TermPremium,RiskPremium,mom%change,mo24mma%change,Volatility
0,2012-04,-3.10,11.65,-1.461378,0.881439,205.422784
1,2012-07,0.35,15.49,3.269344,0.910426,311.983378
2,2012-09,0.35,15.35,9.181302,1.108674,268.840724
3,2012-10,0.70,14.48,-2.162382,1.089768,196.672502
4,2012-11,1.10,15.08,-15.594644,0.926755,148.565336
...,...,...,...,...,...,...
117,2022-05,0.67,14.43,-8.125221,0.926215,339.893895
118,2022-06,0.67,14.04,-9.107738,0.842082,328.243477
119,2022-07,0.15,15.32,2.617388,0.869210,537.347807
120,2022-08,0.00,15.42,5.614508,0.922006,495.248034


In [7]:
def get_YM_from_datetime(dt):
    return str(dt.year) + "-" + str(dt.month)

In [8]:
def scale_series(series, mean, std):
    return (series - mean)/std

In [17]:
def PCA_module(df: pd.DataFrame):
    ''' Assert the names and the types of the columns. Assert the cov matrix is all positive. '''
    assert (df.columns[0] == "Date")
    # assert (ptypes.is_datetime64_dtype(df["Date"]))
    assert all(ptypes.is_numeric_dtype(df[col]) for col in df.columns[1:])
    df = df.dropna(axis = "index")
    # assert ((df.cov(numeric_only=True).values > 0).all())

    ''' Get the date range '''
    df = df.sort_values(by= ["Date"], ascending=False)
    print("The date range is :", df["Date"].iat[-1], "to", df["Date"].iat[0])

    ''' Scale the data '''
    for col in df.columns[1:]:
        df[col] = scale_series(df[col], df[col].mean(), df[col].std())
    
    ''' PCA analysis'''
    pca = sklearn.decomposition.PCA(n_components = len(df.columns)-1) # Minka’s MLE is used to guess the dimension.
    pca.fit(df.iloc[:, 1:])
    df_transformed = pd.DataFrame(pca.transform(df.iloc[:, 1:]))
    df_transformed.insert(0, "Date", df["Date"])
    print(df_transformed)




In [18]:
PCA_module(df)

The date range is : 2012-04 to 2022-09
        Date         0         1         2         3         4
0    2012-04 -0.586438 -0.902769 -0.832253 -0.352469 -0.140629
1    2012-07  0.485671 -1.573757 -0.569674  0.701437 -0.373823
2    2012-09  0.121366 -1.724910 -0.303137  0.835909 -0.232162
3    2012-10 -1.487627 -0.695969  0.068032 -0.108122  0.220724
4    2012-11 -1.136205 -0.747639 -0.170094 -0.034078  0.469272
..       ...       ...       ...       ...       ...       ...
117  2022-05 -2.091027 -0.557388 -1.069903 -0.500829  1.109083
118  2022-06 -0.412761 -0.034278 -0.834869 -0.423494  0.268279
119  2022-07  0.873668 -0.501889 -1.403589  0.087746 -0.399080
120  2022-08 -0.073170 -1.099421 -1.193712  0.247515 -0.347245
121  2022-09 -0.580634 -0.236141  1.339359 -2.295425 -1.231621

[122 rows x 6 columns]


: 